In [1]:
from __future__ import print_function
import sqlite3
import paho.mqtt.client as mqtt
import csv
import datetime
import os
import json
import requests
import sys
import msgpack
from kafka import KafkaProducer
from kafka import SimpleProducer, KafkaClient
#basepath = '/root/sftp/volumes/saqn/

In [4]:
import threading, logging, time
import multiprocessing

from kafka import KafkaConsumer, KafkaProducer

kafkaUrl = "swarm-node00.teco.edu"
kafkaUrl = "129.13.170.153"#smartaqnet-dev.teco.edu"
class Producer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.stop_event = threading.Event()
        
    def stop(self):
        self.stop_event.set()

    def run(self):
        print("Starting producer")
        producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')
        print("Producer started")
        while not self.stop_event.is_set():
            print("someThing")
            print(producer.send('Baka', str(time.time())))
            
            
            time.sleep(2)

        producer.close()

class Consumer(multiprocessing.Process):
    def __init__(self):
        
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        
        print("Starting consumer")
        consumer = KafkaConsumer(bootstrap_servers= kafkaUrl + ':9092',
                                 auto_offset_reset='latest',
                                 consumer_timeout_ms=1000)#,value_deserializer=msgpack.unpackb)
        
        print("Consumer started")
        consumer.subscribe(['Baka'])

        while not self.stop_event.is_set():
            for message in consumer:
                print(message)
                
                if self.stop_event.is_set():
                    break

        consumer.close()

In [9]:
tasks = [
        Producer(),
        Consumer()
]
for t in tasks:
        t.start()

        


Starting producer
Starting consumer
Consumer started
Producer started
someThing
ConsumerRecord(topic=u'Baka', partition=0, offset=0, timestamp=1528444664580, timestamp_type=0, key=None, value='1528444664.19', checksum=None, serialized_key_size=-1, serialized_value_size=13)
ConsumerRecord(topic=u'Baka', partition=0, offset=1, timestamp=1528444666581, timestamp_type=0, key=None, value='1528444666.58', checksum=None, serialized_key_size=-1, serialized_value_size=13)
someThing
ConsumerRecord(topic=u'Baka', partition=0, offset=2, timestamp=1528444668585, timestamp_type=0, key=None, value='1528444668.58', checksum=None, serialized_key_size=-1, serialized_value_size=13)
someThing


In [10]:

for t in tasks:
        t.stop()

In [14]:
for t in tasks:
        t.stop()

In [ ]:
docker run \
  --net=host \
  --rm confluentinc/cp-kafka \
  bash -c "seq 42 | kafka-console-producer --broker-list localhost:29092 --topic bar && echo 'Produced 42 messages.'"

In [ ]:
docker run \
 --net=host \
 --rm \
 confluentinc/cp-kafka \
 kafka-console-consumer --bootstrap-server localhost:29092 --topic bar --new-consumer --from-beginning --max-messages 42